In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F 
import time
import random
import datetime
import os
import numpy as np
import torch.optim as optim
import matplotlib.pyplot as plt
from torch import Tensor
import functools
from torchvision import datasets, transforms
from torch.nn.modules.loss import _Loss
from torch.utils.data import DataLoader
import torchvision
from model.drn import drn_d_54, drn_d_base
from model.CBAM import drn_d_CBAM
from model.resnet import ResNet50 as resnet
from load_data import *
from optimizer import Ranger
from collections import Counter
from sklearn.metrics import roc_curve, auc
from sklearn.metrics import recall_score,accuracy_score, precision_score,f1_score


Matplotlib created a temporary config/cache directory at /tmp/matplotlib-05u2qknp because the default path (/home/ubuntu/.config/matplotlib) is not a writable directory; it is highly recommended to set the MPLCONFIGDIR environment variable to a writable directory, in particular to speed up the import of Matplotlib and to better support multiprocessing.
In /home/ubuntu/tensorflow/anaconda3/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: 
The text.latex.preview rcparam was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In /home/ubuntu/tensorflow/anaconda3/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: 
The mathtext.fallback_to_cm rcparam was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In /home/ubuntu/tensorflow/anaconda3/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: Support for setting the 'mathtext.fallback_to_cm' rcParam is dep

PyTorch Version:  1.5.0+cu101
Torchvision Version:  0.6.0+cu101


In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
torch.cuda.set_device(1)
test_size = 0.2
valid_size = 0.5
train_size = 0.5
batch_size = 4
test_bs = 4
random.seed(time.time())

In [4]:
from torch.autograd import Variable
from torch.autograd import Function
from torchvision import models
from torchvision import utils
import cv2
import sys

In [5]:
##Grad_cam
class FeatureExtractor():
    """ Class for extracting activations and 
    registering gradients from targetted intermediate layers """
    def __init__(self,extractor, model, target_layers):
        self.extractor = extractor
        self.model = model
        self.target_layers = target_layers
        self.gradients = []

    def save_gradient(self, grad):
    	self.gradients.append(grad)

    def __call__(self, x):
        outputs = []
        self.gradients = []
        x = x.to(device)
        for name, module in self.model._modules.items():    
            x = module(x)
            if name in self.target_layers:
                x.register_hook(self.save_gradient)
                outputs += [x]
        return outputs, x

class ModelOutputs():
	""" Class for making a forward pass, and getting:
	1. The network output.
	2. Activations from intermeddiate targetted layers.
	3. Gradients from intermeddiate targetted layers. """
	def __init__(self, model, target_layers):
		self.model = model
		self.feature_extractor = FeatureExtractor(self.model.attention, self.model, target_layers)

	def get_gradients(self):
		return self.feature_extractor.gradients

	def __call__(self, x):
		target_activations, output  = self.feature_extractor(x)
		#print(output.shape)
		output = self.model.avgpool(output)
		output = output.view(output.size(0), -1).to(device)
		#print(output.shape)
		output = self.model.fc(output)
		return target_activations, output

def preprocess_image(img):
    #print(img.shape)
    preprocessed_img = img.copy()
    #print(preprocessed_img.shape)
    preprocessed_img = torch.from_numpy(preprocessed_img)
    preprocessed_img.unsqueeze_(0).unsqueeze_(0)
    input = Variable(preprocessed_img, requires_grad = True)
    return input

def crop_center(img,cropx,cropy):
    y,x,c = img.shape
    startx = x//2 - cropx//2
    starty = y//2 - cropy//2    
    return img[starty:starty+cropy, startx:startx+cropx, :]

def show_cam_on_image(img, mask, num):
    heatmap = cv2.applyColorMap(np.uint8(255*mask), cv2.COLORMAP_JET)
    heatmap = np.float32(heatmap) / 255
    #print(img.shape)
    img = np.transpose(img,(1,2,0))
    cam = heatmap + np.float32(img)
    cam = cam / np.max(cam)
    cv2.imwrite(str(num) + 'dicom.jpg', np.uint8(255 * img))
    img1 = crop_center(img, 400, 400)
    cv2.imwrite(str(num) + 'dicom400.jpg', np.uint8(255 * img1))
    img2 = crop_center(img, 350, 350)
    cv2.imwrite(str(num) + 'dicom350.jpg', np.uint8(255 * img2))
    cv2.imwrite(str(num) + "_cam.jpg", np.uint8(255 * cam))


class GradCam:
    def __init__(self, model, target_layer_names):
        self.model = model
        self.model.eval()  
        self.extractor = ModelOutputs(self.model, target_layer_names)  
    def forward(self, input):
        return self.model(input)   
    def __call__(self, input, index = None):
        
        features, output = self.extractor(input)     
        if index == None:
            index = np.argmax(output.cpu().data.numpy())        
        one_hot = np.zeros((1, output.size()[-1]), dtype = np.float32)
        one_hot[0][index] = 1
        one_hot = Variable(torch.from_numpy(one_hot), requires_grad = True)
        
        one_hot = torch.sum(one_hot.to(device) * output)     
        self.model.drn.zero_grad()
        self.model.fc.zero_grad()
        one_hot.backward()
        #print(len(self.extractor.get_gradients()))
        grads_val = self.extractor.get_gradients()[-1].cpu().data.numpy()        
        target = features[-1]
        target = target.cpu().data.numpy()[0, :]
        #print(target.shape)
        weights = grads_val[0, :]
        #weights = np.mean(grads_val, axis = (2, 3))[0, :]
        cam = np.zeros(target.shape[1 : ], dtype = np.float32)       
        for i, w in enumerate(weights):
        	cam += w * target[i, :, :]     
        cam = np.maximum(cam, 0)
        cam = cv2.resize(cam, (512, 512))
        cam = cam - np.min(cam)
        cam = cam / np.max(cam)
        return cam

class GuidedBackpropReLU():

    def forward(self, input):
        positive_mask = (input > 0).type_as(input)
        output = torch.addcmul(torch.zeros(input.size()).type_as(input), input, positive_mask)
        self.save_for_backward(input, output)
        return output

    def backward(self, grad_output):
        input, output = self.saved_tensors
        grad_input = None

        positive_mask_1 = (input > 0).type_as(grad_output)
        positive_mask_2 = (grad_output > 0).type_as(grad_output)
        grad_input = torch.addcmul(torch.zeros(input.size()).type_as(input), torch.addcmul(torch.zeros(input.size()).type_as(input), grad_output, positive_mask_1), positive_mask_2)

        return grad_input

class GuidedBackpropReLUModel:
	def __init__(self, model):
		self.model = model
		self.model.eval()
		

		# replace ReLU with GuidedBackpropReLU
		for idx, module in self.model.drn._modules.items():
			if module.__class__.__name__ == 'relu':
				self.model.features._modules[idx] = GuidedBackpropReLU()

	def forward(self, input):
		return self.model(input)

	def __call__(self, input, index = None):
		
		output = self.forward(input.to(device))

		if index == None:
			index = np.argmax(output.cpu().data.numpy())

		one_hot = np.zeros((1, output.size()[-1]), dtype = np.float32)
		one_hot[0][index] = 1
		one_hot = Variable(torch.from_numpy(one_hot), requires_grad = True)
		
		one_hot = torch.sum(one_hot.to(device) * output)

		# self.model.features.zero_grad()
		# self.model.classifier.zero_grad()
		one_hot.backward()

		output = input.grad.cpu().data.numpy()
		output = output[0,:,:,:]

		return output



class Hswish(nn.Module):
    def forward(self, x):
        swish = F.relu6(x + 3 , inplace = True)
        return x* swish/6.

class conv_set(nn.Module):
    """docstring for conv_set"""
    def __init__(self, in_ch, out_ch, kernel_size, stride, padding):
        super(conv_set, self).__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(in_ch, out_ch, kernel_size, stride, padding, bias = False),
            nn.BatchNorm2d(out_ch),
            Hswish()
            )
    def forward(self, x):
        out = self.conv(x)
        
        return out

In [7]:
class Focal_Loss(_Loss):
    def __init__(self, alpha = 1 , gamma = 1 , logits = False, reduce = False, ignore_index=10000, from_logits=False):
        super(Focal_Loss, self).__init__()
        self.alpha = alpha
        self.gamma = gamma
        self.ignore_index = ignore_index
        self.logits = logits
        self.reduce = reduce
        self.loss  = nn.BCELoss(weight = torch.Tensor([3.0, 1.0]).to(device))
    def forward(self, inputs, targets):
        #CE = F.cross_entropy(inputs, targets, weight = torch.Tensor([1.0, 4.0]).to(device), ignore_index=self.ignore_index)
        inputs = torch.sigmoid(inputs)
        #print(inputs.shape, targets.shape)
        CE = self.loss(inputs, targets)
        pt = torch.exp(-CE)
        F_loss = self.alpha * (1-pt)**self.gamma * CE

        if self.reduce:
            return torch.mean(F_loss)
        else:
            return F_loss

class Hswish(nn.Module):
    def forward(self, x):
        swish = F.relu6(x + 3 , inplace = True)
        return x* swish/6.

class conv_set(nn.Module):
    """docstring for conv_set"""
    def __init__(self, in_ch, out_ch, kernel_size, stride, padding):
        super(conv_set, self).__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(in_ch, out_ch, kernel_size, stride, padding, bias = False),
            nn.BatchNorm2d(out_ch),
            Hswish()
            )
    def forward(self, x):
        out = self.conv(x)
        return out

class A_net(nn.Module):

    def __init__(self, classes):
        super(A_net, self).__init__()
        self.classes = classes
        self.conv  = nn.Conv2d(1, 3, 1, 1, 0)
        self.conv1 = conv_set(3,12,3,1,0)
        self.conv2 = conv_set(12,16,3,1,0)
        self.conv3 = conv_set(16,32,3,1,0)
        self.conv4 = conv_set(32,classes,3,1,2)
        #self.conv5 = conv_set(64,classes,3,1,0)
    def forward(self, x):
        x = self.conv(x)
        out = self.conv1(x)
        out = F.max_pool2d(out, 2,2)
        out = self.conv2(out)
        out = F.max_pool2d(out, 2,2)
        out = self.conv3(out)
        out = F.max_pool2d(out, 2,2)
        out = self.conv4(out)
        return out 
        
class CNNX(nn.Module):
    
    def __init__(self, backbone = 'drn', out_stride = 16, num_class = 2):
        super(CNNX, self).__init__()
        
        if backbone == 'drn':
            output_stride = 8
        #self.drn = drn_d_base(nn.BatchNorm2d)
        #self.drn = gc_drn_54(nn.BatchNorm2d)
        #self.model = mixnet_l()
        self.model = resnet()
        #self.model = drn_d_CBAM(nn.BatchNorm2d)
        self.attention = A_net(512)
        self.conv_out = nn.Conv2d(512,2,1)
        self.maxpool = nn.AdaptiveAvgPool2d(1)
        self._dropout = nn.Dropout(0.25)
        self.fc = nn.Linear(2048, 2)
        self.fc = nn.Linear(2048, 2)
        


    def forward(self, input):
        x = self.model(input)
       #atten = self.attention(input)
       #atten = torch.sigmoid(atten)
       #x = torch.mul(x, atten)
        #x = self.conv_out(x)
        x = self.maxpool(x)
        x = x.view(x.size(0), -1)
        x = self.fc(x)
        x = self._dropout(x)

        return x


def save_model(state,save_model_path,modelname):
    filename = os.path.join(save_model_path,modelname+'_'+str(state['epoch']+1)+'.pth')
    torch.save(state,filename)

def save_best(state,save_model_path,modelname):
    filename = os.path.join(save_model_path,modelname+'_'+'.pth')
    torch.save(state,filename)

def load_model(Net, optimizer, model_file):
    assert os.path.exists(model_file),'There is no model file from'+model_file
    checkpoint = torch.load(model_file)
    print('load', model_file)
    Net.load_state_dict(checkpoint['model_state_dict'])
    start_epoch = checkpoint['epoch']+1
    if optimizer is not None:
        optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    return Net, optimizer, start_epoch

def train(model, data_loader, opt, loss, epoch,verbose = True):
    model.train()
    loss_avg = 0.0
    correct = 0
    for batch_idx, (data,target) in enumerate(data_loader):
        #print(data.shape)
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = loss_fn(output, target)
        loss_avg = loss_avg + loss.item()
        loss.backward()
        optimizer.step()
        pred = output.data.max(1, keepdim=True)[1] # get the index of the max log-probability
        target = target[:,1].long()
        correct += pred.eq(target.data.view_as(pred)).cpu().sum().item()
        verbose_step = len(data_loader) 
        if (batch_idx+1)  % verbose_step == 0 and verbose:
            print('Train Epoch: {}  Step [{}/{} ({:.0f}%)]  Loss: {:.6f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
                epoch, batch_idx * len(data), len(train_idx),
                100. * batch_idx / len(train_idx), loss.item(), correct, len(train_idx),
            100. * correct / (len(train_idx))))
    return loss_avg / (len(train_idx))

def test(model, data_loader, loss):
    with torch.no_grad():
        model.eval()
        test_loss = 0
        correct = 0
        correct_pe = 0
        correct_normal = 0
        y = []
        y_score = []
        y_pred = []
        y_score = []
        for data, target in data_loader:
            data, target = data.to(device), target.to(device)
            #print(target.shape)
            output = model(data)
            print(output, target)
            test_loss += loss_fn(output, target).item() # sum up batch loss
            pred = output.data.max(1, keepdim=True)[1] # get the index of the max log-probability
            target = target[:,1].long()
            correct += pred.eq(target.data.view_as(pred)).cpu().sum().item()
            score = F.softmax(output, dim = 1)
            for i in range(target.shape[0]):
                y.append(target[i].cpu().numpy())
                y_score.append(score[i,1].cpu().numpy())
                y_pred.append(pred[i].cpu().numpy())
                if target[i] == 1:
                    correct_pe += pred[i].eq(target[i].data.view_as(pred[i])).cpu().sum().item()
    
                if target[i] == 0:
                    correct_normal += pred[i].eq(target[i].data.view_as(pred[i])).cpu().sum().item()

        test_loss /= len(data_loader.dataset)
        print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%), PE_acc: ({}), normal_acc: ({})\n'.format(
            test_loss, correct, len(data_loader.dataset),
            100. * correct / len(data_loader.dataset), correct_pe, correct_normal))   
    #return float(correct) / len(data_loader.dataset),y, y_score, y_pred
    return float(correct) / len(data_loader.dataset)


net = CNNX().to(device)

class TripletLoss(nn.Module):
    """Triplet loss with hard positive/negative mining.
    Reference:
    Hermans et al. In Defense of the Triplet Loss for Person Re-Identification. arXiv:1703.07737.
    Code imported from https://github.com/Cysu/open-reid/blob/master/reid/loss/triplet.py.
    Args:
        margin (float): margin for triplet.
    """

    def __init__(self, margin=0.6, mutual_flag=False):
        super(TripletLoss, self).__init__()
        self.margin = margin
        self.ranking_loss = nn.MarginRankingLoss(margin=margin)
        self.mutual = mutual_flag

    def forward(self, inputs, targets):
        """
        Args:
            inputs: feature matrix with shape (batch_size, feat_dim)
            targets: ground truth labels with shape (num_classes)
        """
        n = inputs.size(0)
        # inputs = 1. * inputs / (torch.norm(inputs, 2, dim=-1, keepdim=True).expand_as(inputs) + 1e-12)
        # Compute pairwise distance, replace by the official when merged
        dist = torch.pow(inputs, 2).sum(dim=1, keepdim=True).expand(n, n)
        dist = dist + dist.t()
        dist.addmm_(1, -2, inputs, inputs.t())
        dist = dist.clamp(min=1e-12).sqrt()  # for numerical stability
        # For each anchor, find the hardest positive and negative
        targets = targets[:,1]
        mask = targets.expand(n, n).eq(targets.expand(n, n).t())
        dist_ap, dist_an = [], []
        for i in range(n):
            dist_ap.append(dist[i][mask[i]].max().unsqueeze(0))
            dist_an.append(dist[i][mask[i] == 0].min().unsqueeze(0))
        dist_ap = torch.cat(dist_ap)
        dist_an = torch.cat(dist_an)
        # Compute ranking hinge loss
        y = torch.ones_like(dist_an)
        loss = self.ranking_loss(dist_an, dist_ap, y)
        if self.mutual:
            return loss, dist
        return loss

class remix_loss(nn.Module):
    """docstring for remix_loss"""
    def __init__(self):
        super(remix_loss, self).__init__()
        self.loss1 = Focal_Loss()
        self.loss2 = TripletLoss()
    def forward(self, inputs, targets):
        loss = self.loss1(inputs, targets) + self.loss2(inputs, np.argmax(targets))

        return loss 


In [8]:
import glob

folder_name =list(glob.glob('/home/ubuntu/Andrew/CT_CLA/20210310/*'))
print(folder_name)

['/home/ubuntu/Andrew/CT_CLA/20210310/1', '/home/ubuntu/Andrew/CT_CLA/20210310/t0', '/home/ubuntu/Andrew/CT_CLA/20210310/v0', '/home/ubuntu/Andrew/CT_CLA/20210310/v1', '/home/ubuntu/Andrew/CT_CLA/20210310/t1', '/home/ubuntu/Andrew/CT_CLA/20210310/0']


In [13]:
num_pic_1 = len(list(glob.glob('/home/ubuntu/Andrew/CT_CLA/1/*')))
num_pic_0 = len(list(glob.glob('/home/ubuntu/Andrew/CT_CLA/lung_contrast+_normal/*')))
print(num_pic_0,num_pic_1)

0 167


In [14]:
'''
split_v_0 = int(valid_size*num_pic_0)
split_v_1 = int(valid_size*num_pic_1)
split_t_0 = int(test_size*num_pic_0)
split_t_1 = int(test_size*num_pic_1)
train_0 = list(glob.glob('/home/ubuntu/Andrew/CT_CLA/20210310/0/*'))
train_1 = list(glob.glob('/home/ubuntu/Andrew/CT_CLA/20210310/1/*'))
valid_0 = list(glob.glob('/home/ubuntu/Andrew/CT_CLA/20210310/v0/*'))
valid_1 = list(glob.glob('/home/ubuntu/Andrew/CT_CLA/20210310/v1/*'))
test_0 = list(glob.glob('/home/ubuntu/Andrew/CT_CLA/20210310/t0/*'))
test_1 = list(glob.glob('/home/ubuntu/Andrew/CT_CLA/20210310/t1/*'))
'''

"\nsplit_v_0 = int(valid_size*num_pic_0)\nsplit_v_1 = int(valid_size*num_pic_1)\nsplit_t_0 = int(test_size*num_pic_0)\nsplit_t_1 = int(test_size*num_pic_1)\ntrain_0 = list(glob.glob('/home/ubuntu/Andrew/CT_CLA/20210310/0/*'))\ntrain_1 = list(glob.glob('/home/ubuntu/Andrew/CT_CLA/20210310/1/*'))\nvalid_0 = list(glob.glob('/home/ubuntu/Andrew/CT_CLA/20210310/v0/*'))\nvalid_1 = list(glob.glob('/home/ubuntu/Andrew/CT_CLA/20210310/v1/*'))\ntest_0 = list(glob.glob('/home/ubuntu/Andrew/CT_CLA/20210310/t0/*'))\ntest_1 = list(glob.glob('/home/ubuntu/Andrew/CT_CLA/20210310/t1/*'))\n"

In [15]:
CT_0 = list(glob.glob('/home/ubuntu/Andrew/CT_CLA/0/*'))
CT_1 = list(glob.glob('/home/ubuntu/Andrew/CT_CLA/1/*'))
random.shuffle(CT_0)
random.shuffle(CT_1)
split_t_0 = int(train_size*num_pic_0)
split_t_1 = int(train_size*num_pic_1)
split_v_0 = int(valid_size*num_pic_0)
split_v_1 = int(valid_size*num_pic_1)
split_te_0 = int(test_size*num_pic_0)
split_te_0 = int(test_size*num_pic_0)
print(split_t_0,split_t_1,split_v_0,split_v_1)
train_0 = CT_0[0:split_t_0]
train_1 = CT_1[0:split_t_1]
valid_0 = CT_0[split_t_0:split_t_0+split_v_0]
valid_1 = CT_1[split_t_1:split_t_1+split_v_1]
test_0 = CT_0[split_t_0+split_v_0:]
test_1 = CT_0[split_t_1+split_v_1:]
#test_0 = list(glob.glob('/home/ubuntu/Andrew/CT_CLA/v0/*'))
#test_1 = list(glob.glob('/home/ubuntu/Andrew/CT_CLA/v1/*'))
print(len(train_0),len(train_1),len(valid_0),len(valid_1),len(test_0),len(test_1))

0 83 0 83
0 83 0 83 204 38


In [16]:
def get_data(folder0, folder1, repeat):
    class0_path = []
    class1_path = []
    target0 = []
    target1 = []
    for x in folder0:
        #print(x)
        class0 = x
        #print(class0)
        class0_path.append(class0)
        target0.append(torch.tensor([1.0,0.0]).long())
    for x in folder1:
        #class1 = glob.glob(x + '/*dcm')
        class1 = x
        class1_path.append(class1)
        target1.append(torch.tensor([0.0,1.0]).long())

    #class0_path = random.sample(class0_path, k = len(class1_path))
    #target0 = random.sample(target0, k = len(target1))
    print(len(class0_path), len(class1_path))
    file_path = class0_path + class1_path
    target = target0 + target1
    #print(file_path, target)
    data = zip(file_path, target)
    return data, len(class0_path), len(class1_path)

In [17]:
train_data, _, _ = get_data(train_0, train_1, 1)
valid_data,pe0, pe1 = get_data(valid_0, valid_1, 1)
test_data, _, _ = get_data(test_0, test_1, 1)

0 83
0 83
204 38


In [18]:
Train_Set = PE(list(train_data), transform = None)
print(Train_Set)
Valid_Set = PE(list(valid_data), transform = None)
print(Valid_Set)
Test_Set = PE(list(test_data), transform = None)

In [19]:
num_train = len(list(Train_Set))
num_valid = len(list(Valid_Set))
indices = list(range(num_train))
v_indices = list(range(num_valid))
train_idx, valid_idx = indices, v_indices

<class 'str'>
[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
<class 'str'>
[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
<class 'str'>
[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
<class 'str'>
[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
<class 'str'>
[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
<class 'str'>
[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
<class 'str'>
[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
<class 'str'>
[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ..

[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
<class 'str'>
[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
<class 'str'>
[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
<class 'str'>
[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
<class 'str'>
[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
<class 'str'>
[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
<class 'str'>
[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
<class 'str'>
[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...


In [20]:
train_loader = DataLoader(Train_Set, batch_size=batch_size, shuffle = True,
                        num_workers=8, pin_memory=True)
valid_loader = DataLoader(Valid_Set, batch_size=test_bs, shuffle = False,
                      num_workers=8, pin_memory=True)
test_loader = DataLoader(Test_Set, batch_size=test_bs, shuffle = False,
                      num_workers=8, pin_memory=True)

In [ ]:

class Hswish(nn.Module):
    def forward(self, x):
        swish = F.relu6(x + 3 , inplace = True)
        return x* swish/6.

class conv_set(nn.Module):
    """docstring for conv_set"""
    def __init__(self, in_ch, out_ch, kernel_size, stride, padding):
        super(conv_set, self).__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(in_ch, out_ch, kernel_size, stride, padding, bias = False),
            nn.BatchNorm2d(out_ch),
            Hswish()
            )
    def forward(self, x):
        out = self.conv(x)
        return out

class A_net(nn.Module):

    def __init__(self, classes):
        super(A_net, self).__init__()
        self.classes = classes
        self.conv  = nn.Conv2d(1, 3, 1, 1, 0)
        self.conv1 = conv_set(3,12,3,1,0)
        self.conv2 = conv_set(12,16,3,1,0)
        self.conv3 = conv_set(16,32,3,1,0)
        self.conv4 = conv_set(32,128,3,1,0)
        self.conv5 = conv_set(128,classes,3,1,0)
    def forward(self, x):
        x = self.conv(x)
        out = self.conv1(x)
        out = F.max_pool2d(out, 2,2)
        out = self.conv2(out)
        out = F.max_pool2d(out, 2,2)
        out = self.conv3(out)
        out = F.max_pool2d(out, 2,2)
        out = self.conv4(out)
        out = self.conv5(out)
        out = F.max_pool2d(out, 2,2)
        return out 
        
class CNNX(nn.Module):
    
    def __init__(self, backbone = 'drn', out_stride = 16, num_class = 2):
        super(CNNX, self).__init__()
        
        if backbone == 'drn':
            output_stride = 8
        self.drn = drn_d_54(nn.BatchNorm2d)
        self.attention = A_net(512)
        self.avgpool = nn.AvgPool2d(36, stride = 1)
        self.fc = nn.Linear(430592, 2)
        


    def forward(self, input):
        x = self.drn(input)
        x = self.avgpool(x)
        
        atten = self.attention(input)
        atten = torch.sigmoid(atten)
        x = torch.mul(x, atten)
        x = x.view(x.size(0), -1)
        x = self.fc(x)

        return x



In [28]:

def load_model(Net, optimizer, model_file):
    assert os.path.exists(model_file),'There is no model file from'+model_file
    checkpoint = torch.load(model_file, map_location='cuda:0')
    Net.load_state_dict(checkpoint['model_state_dict'])
    start_epoch = checkpoint['epoch']+1
    if optimizer is not None:
        optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    return Net, optimizer, start_epoch

net = CNNX().to(device)
#net = CNNX().cpu()
resume = '/home/ubuntu/Andrew/PE_research/log/savemodel/CT_class_resbest_acc_A_.pth'
#resume = 'best_acc_142.pth'
Net, _, _ = load_model(net, None, resume)

if __name__ == '__main__':
    """ python grad_cam.py <path_to_image>
    1. Loads an image with opencv.
    2. Preprocesses it for VGG19 and converts to a pytorch variable.
    3. Makes a forward pass to find the category index with the highest score,
    and computes intermediate activations.
    Makes the visualization. """    

    import glob
    file_name = list(glob.glob('/home/ubuntu/Andrew/CT_CLA/v1/*'))
    file_name_sample = random.sample(file_name, k = 1)
    grad_cam = GradCam(model = Net, \
    				target_layer_names = ["layer8"]) 
    num = 1
    for i in file_name_sample:
        print(i)
        
        img = pydicom.dcmread(i).pixel_array
        img = np.float32(img) / 4095
        input = (img*2)-1
        input = preprocess_image(input)
        # If None, returns the map for the highest scoring category.
        # Otherwise, targets the requested index.
        target_index = None 
        mask = grad_cam(input, target_index)
        img = torch.tensor(img).float()
        img_rgb = img.unsqueeze_(0).repeat(3, 1, 1)    
        show_cam_on_image(img_rgb, mask, num)
        
        gb_model = GuidedBackpropReLUModel(model = Net)
        gb = gb_model(input, index=target_index)
        #utils.save_image(torch.from_numpy(gb), str(num) + '_gb.jpg')    
        cam_mask = np.zeros(gb.shape)
        for i in range(0, gb.shape[0]):
            cam_mask[i, :, :] = mask.transpose(1,0)    
        cam_gb = np.multiply(cam_mask, gb)
        utils.save_image(torch.from_numpy(cam_gb), str(num) + '_cam_gb.jpg')
        num += 1

In [23]:
learning_rate = 0.0001
optimizer = optim.SGD(net.parameters(), lr=learning_rate, momentum=0.9, weight_decay=1e-4)
#optimizer = Ranger(net.parameters())

# Train the model
num_epochs = 50
#model_name = 'resnet9'
model_name = 'drn'
#loss_fn = nn.BCELoss()
loss_fn = Focal_Loss()
output_path = 'log'
resume = 'None'

#resume = '/home/ubuntu/Andrew/PE_research/log/savemodel/0428_resbest_acc_.pth'
#resume = '/home/ubuntu/Andrew/PE_research/log/savemodel/CT_class_resbest_acc_.pth'
StartTime = time.time()
loss, val_acc, lr_curve = [], [], []
'''
if resume is not None:
    net, optimizer, _ = load_model(net, optimizer, resume)
    #print('load', resume)

'''
best_acc = 0.0
best = 0
for epoch in range(num_epochs):

    
    #if (epoch+1)%5 == 0:
    #    random.seed(epoch)
    #    train_data = get_train_data(train_folder)
    #    Train_Set = PE(list(train_data), transform = None)
    #    train_loader = DataLoader(Train_Set, batch_size=batch_size, shuffle = True,
    #                    num_workers=8, pin_memory=True)
    #    class0_path = random.sample(class0_path, k = 3*len(class1_path))
    #    target0 = random.sample(target0, k = 3*len(target1))
    #    print(len(class0_path), len(class1_path))
    #    file_path = class0_path + class1_path
    #    target = target0 + target1
    #    #print(file_path, target)
    #    train_data = zip(file_path, target)
    #    Train_Set = PE(list(train_data), transform = True)
    #    train_loader = DataLoader(Train_Set, batch_size=batch_size, shuffle = True,
    #                    num_workers=4, pin_memory=True)
    #lr = adjust_learning_rate(learning_rate, optimizer, epoch, epoch_list=[80, 170])
    train_loss = train(net, train_loader, optimizer, loss_fn, epoch, verbose=True)
    valid_acc = test(net, valid_loader, loss_fn)
    #valid_acc, tp, tn = test(net, valid_loader, loss_fn)
    loss.append(train_loss)
    #print((tn/pe0), tp/pe1)
    val_acc.append(valid_acc)
    if (epoch+1)%10 == 0 or epoch==num_epochs-1:
        save_model({'epoch':epoch,
                    'model_state_dict':net.state_dict(),
                    'optimizer_state_dict':optimizer.state_dict(),
                    },
                    os.path.join(output_path,'savemodel'),model_name)
    
    #if tp/pe1 > 0.7 and tn>best:
    if valid_acc >= best_acc:
        #print((tn//pe0), tp//pe1)
        print(valid_acc)
        save_best({'epoch':epoch,
                    'model_state_dict':net.state_dict(),
                    'optimizer_state_dict':optimizer.state_dict(),
                    },
                    os.path.join(output_path,'savemodel'),'CT_class_resbest_acc_A1')
        best_acc = valid_acc
        #best = tn
    





<class 'str'>
<class 'str'><class 'str'><class 'str'><class 'str'><class 'str'><class 'str'>

<class 'str'>



[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]

[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]][[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]][[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]][[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]][[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]][[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]][[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn import manifold, datasets

In [ ]:
class PE_TSNE(Dataset):
    def __init__(self, file, transform=None):
        self.data = file
        self.transform = transform
        

    def __getitem__(self, index):

        data = self.data

        #img, target = dicom_loader(data)
        
        #img = crop_center(img, 400, 400)
        #print(img)
        return data

    def __len__(self):
        return len(self.data)

In [ ]:
Train_TSNE = PE_TSNE(list(train_data), transform = None)
print(Train_TSNE)
Valid_TSNE = PE_TSNE(list(valid_data), transform = None)
print(Valid_TSNE)
Test_TSNE = PE_TSNE(list(test_data), transform = None)
print(Test_TSNE)

In [ ]:
def get_data_TSNE(folder0, folder1, repeat):
    class0_path = []
    class1_path = []
    target0 = []
    target1 = []
    for x in folder0:
        #print(x)
        class0 = x
        #print(class0)
        class0_path.append(class0)
        target0.append(torch.tensor([1.0,0.0]).long())
    for x in folder1:
        #class1 = glob.glob(x + '/*dcm')
        class1 = x
        class1_path.append(class1)
        target1.append(torch.tensor([0.0,1.0]).long())

    #class0_path = random.sample(class0_path, k = len(class1_path))
    #target0 = random.sample(target0, k = len(target1))
    print(len(class0_path), len(class1_path))
    file_path = class0_path + class1_path
    target = target0 + target1
    #print(file_path, target)
    data = file_path
    return data, len(class0_path), len(class1_path)

In [11]:
test_data, _, _ = get_data(test_0, test_1, 1)

NameError: name 'test_0' is not defined

In [ ]:
'''t-SNE'''
tsne = manifold.TSNE(n_components=2, init='pca', random_state=501)
X_tsne = tsne.fit_transform(Test_TSNE)

print("Org data dimension is {}.Embedded data dimension is {}".format(X.shape[-1], X_tsne.shape[-1]))

'''嵌入空间可视化'''
x_min, x_max = X_tsne.min(0), X_tsne.max(0)
X_norm = (X_tsne - x_min) / (x_max - x_min)  # 归一化
plt.figure(figsize=(8, 8))
for i in range(X_norm.shape[0]):
    plt.text(X_norm[i, 0], X_norm[i, 1], str(y[i]), color=plt.cm.Set1(y[i]), 
             fontdict={'weight': 'bold', 'size': 9})
plt.xticks([])
plt.yticks([])
plt.show()

In [ ]:
plt.figure()
plt.plot(loss)
plt.title('Train Loss')
plt.xlabel('epochs')
plt.ylabel('loss')
plt.show()
plt.figure()

plt.plot(valid_acc)
plt.title('accuracy')
plt.xlabel('epochs')
plt.ylabel('acc')
plt.show()

plt.figure()
plt.plot(val_acc)

plt.title('Valid Acc')
plt.xlabel('epochs')
plt.ylabel('accuracy')
plt.show()

In [21]:
test_0 = list(glob.glob('/home/ubuntu/Andrew/CT_CLA/v0/*'))
test_1 = list(glob.glob('/home/ubuntu/Andrew/CT_CLA/v1/*'))
test_data, _, _ = get_data(test_0, test_1, 1)
Test_Set = PE(list(test_data), transform = None)
test_loader = DataLoader(Test_Set, batch_size=test_bs, shuffle = False,
                      num_workers=8, pin_memory=True)

56 23


In [25]:
optimizer = optim.SGD(net.parameters(), lr=learning_rate, momentum=0.9, weight_decay=1e-4)
net, _, _ = load_model(net, optimizer, '/home/ubuntu/Andrew/PE_research/log/savemodel/CT_class_resbest_acc_A1_.pth')
test_acc = test(net, test_loader, loss_fn)
net, _, _ = load_model(net, optimizer, '/home/ubuntu/Andrew/PE_research/log/savemodel/drn_10.pth')
test_acc = test(net, test_loader, loss_fn)
net, _, _ = load_model(net, optimizer, '/home/ubuntu/Andrew/PE_research/log/savemodel/drn_20.pth')
test_acc = test(net, test_loader, loss_fn)
#test_acc, y, y_score, y_pred = test(net, test_loader, loss_fn)
'''
y = np.array(y)
y_pred = np.array(y_pred)
y_score = np.array(y_score)

precision = precision_score(y, y_pred, average='weighted')
recall = recall_score(y, y_pred, average='weighted')
print('precision', precision, 'recall', recall)
ftr, tpr, _ = roc_curve(y, y_pred)
roc_auc = auc(ftr, tpr)
print('auc', roc_auc)
'''
EndTime = time.time()
print('Time Usage: ', str(datetime.timedelta(seconds=int(round(EndTime-StartTime)))))



plt.figure()
plt.plot(loss)
plt.title('Train Loss')
plt.xlabel('epochs')
plt.ylabel('loss')
plt.show()
plt.figure()

plt.plot(valid_acc)
plt.title('accuracy')
plt.xlabel('epochs')
plt.ylabel('acc')
plt.show()

plt.figure()
plt.plot(val_acc)
plt.title('Valid Acc')
plt.xlabel('epochs')
plt.ylabel('accuracy')
plt.show()

load /home/ubuntu/Andrew/PE_research/log/savemodel/CT_class_resbest_acc_.pth
set state called
load /home/ubuntu/Andrew/PE_research/log/savemodel/drn_20.pth
set state called

Test set: Average loss: 0.8557, Accuracy: 60/79 (76%), PE_acc: (4), normal_acc: (56)

In [29]:
net, _, _ = load_model(net, optimizer, '/home/ubuntu/Andrew/PE_research/log/savemodel/CT_class_resbest_acc_A1_.pth')
print("1")
test_acc = test(net, test_loader, loss_fn)
print("2")